In [1]:
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
import joblib
from os import walk

import time

%matplotlib inline

In [2]:
#pathBase = 'I://Chuan//Documents//MyData//DICOM//NormalCAG//Arrangement//'
pathBase = 'E://Chuan//Documents//Database//DICOM//CoroBase//Arrangement//'

theWhole = {}

f = []
f_LCA_Caudal = []
mypath_LCA_Caudal = pathBase + 'LCA_Caudal'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Caudal):
    f.extend(filenames)
for l in f:
    f_LCA_Caudal.append(mypath_LCA_Caudal + '\\' + l) 
    
f = []
f_LCA_Cranial = []
mypath_LCA_Cranial = pathBase + 'LCA_Cranial'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial.append(mypath_LCA_Cranial + '\\' + l) 
    
f = []
f_LCA_Cranial_LAO = []
mypath_LCA_Cranial_LAO = pathBase + 'LCA_Cranial_LAO'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial_LAO):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial_LAO.append(mypath_LCA_Cranial_LAO + '\\' + l) 
    
f = []
f_LCA_Spider = []
mypath_LCA_Spider = pathBase + 'LCA_Spider'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Spider):
    f.extend(filenames)
for l in f:
    f_LCA_Spider.append(mypath_LCA_Spider + '\\' + l) 
    
f = []
f_RCA_Cranial = []
mypath_RCA_Cranial = pathBase + 'RCA_Cranial'
for (dirpath, dirnames, filenames) in walk(mypath_RCA_Cranial):
    f.extend(filenames)
for l in f:
    f_RCA_Cranial.append(mypath_RCA_Cranial + '\\' + l) 
    
f = []
f_RCA_LAO = []
mypath_RCA_LAO = pathBase + 'RCA_LAO'
for (dirpath, dirnames, filenames) in walk(mypath_RCA_LAO):
    f.extend(filenames)
for l in f:
    f_RCA_LAO.append(mypath_RCA_LAO + '\\' + l) 
    

theWhole['LCA_Caudal'] = f_LCA_Caudal
theWhole['LCA_Cranial'] = f_LCA_Cranial
theWhole['LCA_Cranial_LAO'] = f_LCA_Cranial_LAO
theWhole['LCA_Spider'] = f_LCA_Spider
theWhole['RCA_Cranial'] = f_RCA_Cranial
theWhole['RCA_LAO'] = f_RCA_LAO
        

In [3]:
theWhole['LCA_Cranial_LAO'][1].split('.')[-1]

'npy'

In [4]:
start = time.time()

array = {}
i = 0

for view in theWhole.keys():
    array[view] = []
    for case in theWhole[view]:
        if case.split('.')[-1] == 'npy':
            arr = np.load(case)
            array[view].append(arr)

end = time.time()
print(end - start)

20.193604230880737


### 2D to 1D

In [5]:
array_flat = {}

for view in array.keys():
    array_flat[view] = []
    for case in array[view]:
        case_flat = []
        for row in case:
            for column in row:
                case_flat.append(column)            
        array_flat[view].append(case_flat)

## Support Vector Classification

In [6]:
x_train_list = []
y_train_list = []

for y, x in array_flat.items():    
    for x_item in x:
        x_train_list.append(x_item)
        y_train_list.append(y)

### Training

In [7]:
start = time.time()

X_train = np.array(x_train_list)
y_train = np.array(y_train_list)

# linear kernel
clf = svm.SVC(kernel='linear', decision_function_shape='ovo')
clf.fit(X_train, y_train)

end = time.time()

In [8]:
print(end - start)

19.393780946731567


### Prediction

In [9]:
print('w = ',clf.coef_)
print('b = ',clf.intercept_)
print('Indices of support vectors = ', clf.support_)
print('Support vectors = ', clf.support_vectors_)
print('Number of support vectors for each class = ', clf.n_support_)
print('Coefficients of the support vector in the decision function = ', np.abs(clf.dual_coef_))

w =  [[ 4.07700569e-06  3.73009381e-06  3.70253982e-06 ... -3.90884262e-06
  -4.00064318e-06 -4.17860405e-06]
 [-6.72017218e-07 -1.09302092e-06 -1.36078007e-06 ... -1.31868588e-06
  -1.59262488e-06 -1.98066127e-06]
 [ 4.20620204e-07  2.16944548e-07  1.97476443e-07 ... -1.57975630e-06
  -1.63231994e-06 -1.70836530e-06]
 ...
 [-2.35279115e-06 -3.11365729e-06 -3.50231949e-06 ... -8.67077112e-07
  -6.52256381e-07 -4.34541802e-07]
 [-2.48786148e-06 -2.31837714e-06 -2.12146951e-06 ... -6.80779275e-08
  -2.40191515e-07 -3.74366081e-07]
 [-1.47030209e-06 -6.98380962e-07 -2.55315576e-07 ... -3.07187296e-07
  -1.12792924e-06 -1.59008221e-06]]
b =  [-2.45185976  0.42973919 -1.18840283 -0.06223153 -0.98161409  4.78625964
  3.12223816  0.61131271 -0.94575853  1.86212618 -3.61986913 -1.88671676
 -5.06802583 -3.26389147 -2.55698988]
Indices of support vectors =  [  0   1   2   3   6   7  10  13  14  18  21  22  23  24  25  28  30  36
  38  40  41  42  44  45  47  48  49  50  51  52  54  55  56  57  6

In [10]:
#pathBase = 'I://Chuan//Documents//MyData//DICOM//NormalCAG//Arrangement//test//'
pathBase = 'E://Chuan//Documents//Database//DICOM//CoroBase//Arrangement//test//'

theWhole = {}

f = []
f_LCA_Caudal = []
mypath_LCA_Caudal = pathBase + 'LCA_Caudal'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Caudal):
    f.extend(filenames)
for l in f:
    f_LCA_Caudal.append(mypath_LCA_Caudal + '\\' + l) 
    
f = []
f_LCA_Cranial = []
mypath_LCA_Cranial = pathBase + 'LCA_Cranial'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial.append(mypath_LCA_Cranial + '\\' + l) 
    
f = []
f_LCA_Cranial_LAO = []
mypath_LCA_Cranial_LAO = pathBase + 'LCA_Cranial_LAO'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Cranial_LAO):
    f.extend(filenames)
for l in f:
    f_LCA_Cranial_LAO.append(mypath_LCA_Cranial_LAO + '\\' + l) 
    
f = []
f_LCA_Spider = []
mypath_LCA_Spider = pathBase + 'LCA_Spider'
for (dirpath, dirnames, filenames) in walk(mypath_LCA_Spider):
    f.extend(filenames)
for l in f:
    f_LCA_Spider.append(mypath_LCA_Spider + '\\' + l) 
    
f = []
f_RCA_Cranial = []
mypath_RCA_Cranial = pathBase + 'RCA_Cranial'
for (dirpath, dirnames, filenames) in walk(mypath_RCA_Cranial):
    f.extend(filenames)
for l in f:
    f_RCA_Cranial.append(mypath_RCA_Cranial + '\\' + l) 
    
f = []
f_RCA_LAO = []
mypath_RCA_LAO = pathBase + 'RCA_LAO'
for (dirpath, dirnames, filenames) in walk(mypath_RCA_LAO):
    f.extend(filenames)
for l in f:
    f_RCA_LAO.append(mypath_RCA_LAO + '\\' + l) 
    

theWhole['LCA_Caudal'] = f_LCA_Caudal
theWhole['LCA_Cranial'] = f_LCA_Cranial
theWhole['LCA_Cranial_LAO'] = f_LCA_Cranial_LAO
theWhole['LCA_Spider'] = f_LCA_Spider
theWhole['RCA_Cranial'] = f_RCA_Cranial
theWhole['RCA_LAO'] = f_RCA_LAO
        

In [11]:
start = time.time()

array = {}
i = 0

for view in theWhole.keys():
    array[view] = []
    for case in theWhole[view]:
        if case.split('.')[-1] == 'npy':
            arr = np.load(case)
            array[view].append(arr)

end = time.time()
print(end - start)

2.233703851699829


### 2D to 1D

In [12]:
array_flat = {}

for view in array.keys():
    array_flat[view] = []
    for case in array[view]:
        case_flat = []
        for row in case:
            for column in row:
                case_flat.append(column)            
        array_flat[view].append(case_flat)

In [13]:
x_test_list = []
y_test_list = []

for y, x in array_flat.items():    
    for x_item in x:
        x_test_list.append(x_item)
        y_test_list.append(y)

In [14]:
X_test = np.array(x_test_list)
y_test = np.array(y_test_list)

## Performance Measures

In [15]:
y_predict = clf.predict(X_test)
y_test = y_test

In [16]:
y_predict

array(['LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Caudal', 'LCA_Caudal', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial', 'RCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Caudal', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial_LAO', 'LCA_Cranial_LAO',
       'LCA_Cranial', 'LCA_Cranial_LAO', 'LCA_Cranial', 'RCA_Cranial',
       'LCA_Spider', 'RCA_Cranial', 'LCA_Spider', 'LCA_Spider',
       'LCA_Spider', 'LCA_Spider', 'LCA_Spider', 'LCA_Spider',
       'LCA_Spider', 'LCA_Spider', 'RCA_Cranial', 'RCA_Cranial',
       'LCA_Cranial_LAO', 'RCA_LAO', 'RCA_Cranial', 'LCA_Cranial',
       'RCA_Cranial', 'RCA_LAO', 'RCA_Cranial', 'RCA_Cranial', 'RCA_LAO',
       'RCA_LAO', 'RCA_LAO', 'RCA_LAO', 'RCA_LAO', 'RCA_LAO', 'RCA_LAO',
       'RCA_LAO', 'RCA_LAO', 'RCA_LAO', 'RCA_LAO'], dtype='<U15')

In [17]:
y_test

array(['LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal', 'LCA_Caudal',
       'LCA_Caudal', 'LCA_Caudal', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial',
       'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial', 'LCA_Cranial_LAO',
       'LCA_Cranial_LAO', 'LCA_Cranial_LAO', 'LCA_Cranial_LAO',
       'LCA_Cranial_LAO', 'LCA_Spider', 'LCA_Spider', 'LCA_Spider',
       'LCA_Spider', 'LCA_Spider', 'LCA_Spider', 'LCA_Spider',
       'LCA_Spider', 'LCA_Spider', 'LCA_Spider', 'RCA_Cranial',
       'RCA_Cranial', 'RCA_Cranial', 'RCA_Cranial', 'RCA_Cranial',
       'RCA_Cranial', 'RCA_Cranial', 'RCA_Cranial', 'RCA_Cranial',
       'RCA_Cranial', 'RCA_LAO', 'RCA_LAO', 'RCA_LAO', 'RCA_LAO',
       'RCA_LAO', 'RCA_LAO', 'RCA_LAO', 'RCA_LAO', 'RCA_LAO', 'RCA_LAO',
       'RCA_LAO'], dtype='<U15')

### Confusion Matrix (混淆矩阵)

In [18]:
cm = np.array(confusion_matrix(y_test, y_predict, labels=['LCA_Caudal', 'LCA_Cranial', 
                                                          'LCA_Cranial_LAO', 'LCA_Spider', 
                                                          'RCA_Cranial', 'RCA_LAO']))
confusion = pd.DataFrame(cm, index=['LCA_Caudal', 'LCA_Cranial', 
                                    'LCA_Cranial_LAO', 'LCA_Spider',
                                    'RCA_Cranial', 'RCA_LAO'],
                        columns=['Predictive LCA_Caudal', 'Predictive LCA_Cranial', 
                                    'Predictive LCA_Cranial_LAO', 'Predictive LCA_Spider',
                                    'Predictive RCA_Cranial', 'Predictive RCA_LAO'])
confusion.to_csv('ConfusionMatrix.csv')
confusion

,Predictive LCA_Caudal,Predictive LCA_Cranial,Predictive LCA_Cranial_LAO,Predictive LCA_Spider,Predictive RCA_Cranial,Predictive RCA_LAO
LCA_Caudal,10,0,0,0,0,0
LCA_Cranial,1,10,1,0,1,0
LCA_Cranial_LAO,0,2,2,0,1,0
LCA_Spider,0,0,0,9,1,0
RCA_Cranial,0,1,1,0,6,2
RCA_LAO,0,0,0,0,0,11


In [19]:
print(classification_report(y_test, y_predict))

                 precision    recall  f1-score   support

     LCA_Caudal       0.91      1.00      0.95        10
    LCA_Cranial       0.77      0.77      0.77        13
LCA_Cranial_LAO       0.50      0.40      0.44         5
     LCA_Spider       1.00      0.90      0.95        10
    RCA_Cranial       0.67      0.60      0.63        10
        RCA_LAO       0.85      1.00      0.92        11

       accuracy                           0.81        59
      macro avg       0.78      0.78      0.78        59
   weighted avg       0.81      0.81      0.81        59



## Save the Model

In [20]:
c = joblib.dump(clf, 'trained_model.m')
print(c)

['trained_model.m']


## Load the Model

In [21]:
d = joblib.load('trained_model.m')
print(d)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
